In [1]:
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

Using Theano backend.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'C:\\Users\\tera2\\AppData\\Local\\Temp\\try_flags_emv6_4sx.c:4:19: fatal error: cudnn.h: No such file or directory\r\ncompilation terminated.\r\n'
Preallocating 716/1024 Mb (0.700000) on cuda0
Mapped name None to device cuda0: GeForce GTX 460 (0000:01:00.0)


In [22]:
env = gym.make('MountainCarContinuous-v0')
env.reset()
goal_steps = 1000
score_requirement = 0
intial_games = 100000

In [23]:
def random_game():
    for step_index in range(10):
        #env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
        print("info: {}".format(info))
    
        if done:
            break
    env.reset()

In [ ]:
random_game()

In [25]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        
        score = 0
        game_memory = []
        previous_observation = []
        
        for step_index in range(goal_steps):
            action = []
            action1 = random.randrange(-1, 2)
            action.append(action1)
            
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1][0] == 0:
                    output = [0, 1, 0]
                elif data[1][0] == -1:
                    output = [1, 0, 0]
                elif data[1][0] == 1:
                    output = [0, 0, 1]
                training_data.append([data[0], output])
        
        env.reset()
    
    print(accepted_scores)
    
    return training_data

In [26]:
training_data = model_data_preparation()

[36.99999999999938, 49.59999999999956, 50.599999999999575, 63.49999999999976, 36.09999999999936, 44.39999999999949, 48.299999999999535, 46.899999999999515, 60.999999999999716, 57.99999999999968, 41.999999999999446, 78.09999999999997, 35.399999999999395, 49.99999999999956, 50.69999999999957, 40.29999999999943, 44.79999999999949, 67.29999999999981, 35.79999999999937, 51.39999999999958, 37.69999999999939, 37.69999999999939, 57.499999999999666, 41.69999999999945, 51.49999999999958, 73.19999999999989, 63.59999999999975, 54.199999999999626, 36.89999999999938, 53.79999999999962, 50.69999999999958, 62.69999999999975, 36.29999999999937, 41.29999999999944, 66.0999999999998, 40.39999999999943, 67.19999999999982, 50.59999999999957, 43.89999999999948, 50.39999999999957, 41.09999999999943, 52.89999999999961, 34.79999999999943, 42.49999999999946, 55.29999999999964, 63.59999999999975, 35.2999999999994, 62.99999999999975, 57.19999999999967, 55.69999999999965, 52.1999999999996, 58.79999999999969, 64.399

In [27]:
def build_model(input_size, output_size):
        model = Sequential()
        model.add(Dense(128, input_dim=input_size, activation='relu'))
        model.add(Dense(52, activation='relu'))
        model.add(Dense(output_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam())

        return model

In [33]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=5)
    return model

In [34]:
trained_model = train_model(training_data)

Epoch 1/5
3648651/3648651 [==============================] - 177s 48us/step - loss: 0.2218
Epoch 2/5
3648651/3648651 [==============================] - 169s 46us/step - loss: 0.2217
Epoch 3/5
3648651/3648651 [==============================] - 169s 46us/step - loss: 0.2217
Epoch 4/5
3648651/3648651 [==============================] - 165s 45us/step - loss: 0.2217
Epoch 5/5
3648651/3648651 [==============================] - 178s 49us/step - loss: 0.2217


In [35]:
model_json = trained_model.to_json()
json_file = open("model_AIGame_MCC_v5.json", "w")
json_file.write(model_json)
json_file.close()
trained_model.save_weights("model_AIGame_MCC_v5.h5")


In [36]:
scores = []
for each_game in range(100):
    score = 0
    prev_obs = []
    action = []
    action.append(1)
    for step_index in range(goal_steps):
        # Uncomment this line if you want to see how our bot playing
        #env.render()
        if len(prev_obs)==0:
            action[0] = random.randrange(-1,1)
        else:
            action[0] = -1 +np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
            #print(action)
        
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score += reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))

[91.70000000000002, 91.80000000000001, 91.90000000000002, 91.90000000000002, 92.20000000000002, 91.70000000000002, 91.90000000000002, 91.80000000000001, 92.20000000000002, 91.80000000000001, 92.10000000000002, 92.00000000000001, 92.00000000000001, 92.00000000000001, 92.20000000000002, 91.90000000000002, 91.40000000000002, 92.10000000000002, 91.90000000000002, 92.00000000000001, 92.10000000000002, 92.10000000000002, 91.90000000000002, 91.90000000000002, 92.00000000000001, 91.90000000000002, 91.70000000000002, 91.70000000000002, 92.00000000000001, 91.40000000000002, 91.80000000000001, 92.00000000000001, 92.00000000000001, 92.00000000000001, 91.90000000000002, 91.30000000000003, 91.90000000000002, 91.90000000000002, 91.90000000000002, 91.40000000000002, 91.40000000000002, 91.60000000000002, 91.90000000000002, 91.20000000000002, 92.00000000000001, 92.10000000000002, 91.70000000000002, 91.80000000000001, 91.80000000000001, 92.00000000000001, 91.30000000000003, 91.90000000000002, 92.00000000